In [ ]:
from tkinter import Tk, Label, Entry, Button, Text
from reportlab.pdfgen import canvas
from tkinter.ttk import *
import os
import webbrowser

def next_line(pos):
    new_pos = pos-20
    
    return new_pos
    
# Function to add task and time to dictionary
def add_task():
  task = task_entry.get()
  time_spent = time_entry.get()
  try:
    # Convert time to integer
    time_spent = int(time_spent)
  except ValueError:
    # Handle invalid input (optional: show error message)
    time_spent = 0
  tasks[task] = time_spent
  task_entry.delete(0, 'end')  # Clear task entry
  time_entry.delete(0, 'end')  # Clear time entry
  update_task_list()

# Function to update the displayed task list
def update_task_list():
  task_list.delete(1.0, 'end')  # Clear existing task list
  if not tasks:
    task_list.insert(1.0, "No tasks added yet!")
    return
  for task, time in tasks.items():
    task_list.insert('end', f"{task}: {time} minutes\n")

def get_sum():
    total = 0
    for task, time in tasks.items():
        total += time
        
    return total

def get_costs(n):
    rates = rate_entry.get()
    rates =  int(rates)
    minute_rate = rates/60
    
    cost = minute_rate * n
    
    return cost
    
def generate_report():
    global tasks
    
    project = project_entry.get()
    
    task_list.insert('end', f"Generating report...")
    
    # Generate PDF report
    report = canvas.Canvas("report.pdf")
    report.setTitle("Time Spent Report")

    # Add headers
    report.drawString(50, 800, f"Project: {project}")
    report.drawString(50, 750, "Task")
    report.drawString(250, 750, "Time (minutes)")
    y_pos = 700

    # Add data for each task
    for task, time in tasks.items():
        report.drawString(50, y_pos, task)
        report.drawString(250, y_pos, str(time))
        y_pos -= 20
    
    # Add total
    total_project_time = get_sum()
    report.drawString(50, y_pos, "Total")
    report.drawString(250, y_pos, str(total_project_time))
    
    # Total Cost
    costs =  get_costs(total_project_time)
    report.drawString(50, y_pos-20, "Project Total Cost")
    report.drawString(250, y_pos-20, "R$ " + str(costs))

    # Save the PDF
    report.save()
    task_list.insert('end',"\nReport succesfully generated!")
    
    webbrowser.open("report.pdf")
    
# Create the main window
root = Tk()
root.title("Time Tracking")

# Set the initial theme
root.tk.call("source", "theme/azure.tcl")
root.tk.call("set_theme", "dark")

# Labels and entries for task and time
project_label = Label(root, text="Project:")
project_label.grid(row=0, column=0)
project_entry = Entry(root)
project_entry.grid(row=0, column=1)

# Hourly cost
rate_label = Label(root, text="Hourly rate:")
rate_label.grid(row=1, column=0)
rate_entry = Entry(root)
rate_entry.grid(row=1, column=1)

tasks = {}  # Dictionary to store tasks and time spent

# Adding tasks and times
task_label = Label(root, text="Task:")
task_label.grid(row=2, column=0)
task_entry = Entry(root)
task_entry.grid(row=2, column=1)

time_label = Label(root, text="Time Spent (minutes):")
time_label.grid(row=3, column=0)
time_entry = Entry(root)
time_entry.grid(row=3, column=1)

# Buttons for adding and printing tasks
add_button = Button(root, text="Add Task", command=add_task)
add_button.grid(row=4, columnspan=2)

#Button for generating report
add_button = Button(root, text="Generate Report", command=generate_report)
add_button.grid(row=5, columnspan=2)

# Text widget to display tasks
task_list = Text(root, height=10, width=50)
task_list.grid(row=6, columnspan=2)

# Update task list initially (optional, shows "No tasks added yet!")
update_task_list()

root.mainloop()